In [ ]:
import numpy as np
import json
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.regularizers import l2
import os

# cd to your folder
data_directory = '/content/drive/MyDrive/Data_Capstone_KashafMujeeb'
file_names = ['realistic_trajectory_data-3.json', 'realistic_trajectory_data-4.json', 'realistic_trajectory_data-5.json', 'realistic_trajectory_data-6.json', 'realistic_trajectory_data-7.json', 'realistic_trajectory_data-8.json', 'realistic_trajectory_data-9.json', 'realistic_trajectory_data-10.json', 'realistic_trajectory_data-11.json', 'realistic_trajectory_data-12.json', 'realistic_trajectory_data-13.json', 'realistic_trajectory_data-14.json', 'realistic_trajectory_data-15.json', 'realistic_trajectory_data-16.json', ]  # Add all your file names here
trajectories = []
initial_accs = []
in_goal = []

# load and combine data from all files
for file_name in file_names:
    file_path = os.path.join(data_directory, file_name)
    with open(file_path, 'r') as file:
        data = json.load(file)
        trajectories.extend([[point['y']] for point in data])
        initial_accs.extend([[point['ay']] for point in data])
        in_goal.extend([1 if point['in_goal'] else 0 for point in data])

num_timesteps = 13  # Number of timesteps per sequence

X_train_pos = []
X_train_acc = []
y_train = []
y_train_goal = []

for i in range(len(trajectories) - num_timesteps):
    X_train_pos.append(trajectories[i:i+num_timesteps])
    X_train_acc.append(initial_accs[i])
    y_train.append(trajectories[i + num_timesteps])
    y_train_goal.append(in_goal[i + num_timesteps])

X_train_pos = np.array(X_train_pos)
X_train_acc = np.array(X_train_acc)
y_train = np.array(y_train)
y_train_goal = np.array(y_train_goal)


In [ ]:
from keras.layers import Dense, LSTM, Dropout, Input, Concatenate, Reshape
from keras.models import Model
from keras.regularizers import l2

# define the LSTM model
input_pos = Input(shape=(num_timesteps, 1))  # Only 1 feature per timestep (x position)
input_acc = Input(shape=(1,))  # Only 1 feature for acceleration (ax), reshaped later

# lSTM layers for trajectory
x = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(input_pos)
x = Dropout(0.2)(x)
x = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)
x = LSTM(64, kernel_regularizer=l2(0.01))(x)

# reshape acceleration input to match the dimensions needed for Concatenate
reshaped_acc = Reshape((1,))(input_acc)
combined = Concatenate(axis=-1)([x, reshaped_acc])

# output layers
output_pos = Dense(1, activation='linear', name='position')(combined)  # Predicts x position
output_goal = Dense(1, activation='sigmoid', name='goal')(combined)  # Predicts goal status

model = Model(inputs=[input_pos, input_acc], outputs=[output_pos, output_goal])

# compile the model
model.compile(optimizer='adam', loss={'position': 'mean_squared_error', 'goal': 'binary_crossentropy'},
              loss_weights={'position': 1.0, 'goal': 0.1})

# train the model
model.fit([X_train_pos, X_train_acc], [y_train, y_train_goal], epochs=200, batch_size=64, validation_split=0.2)

# save the model
model.save('y_position_goal_model.keras')

Epoch 1/200
228/228 [==============================] - 38s 122ms/step - loss: 2950.8645 - position_loss: 2933.5994 - goal_loss: 135.6567 - val_loss: 1627.2349 - val_position_loss: 1621.4207 - val_goal_loss: 24.5869
Epoch 2/200
228/228 [==============================] - 26s 112ms/step - loss: 1627.9346 - position_loss: 1624.6713 - goal_loss: 2.0787 - val_loss: 1582.3024 - val_position_loss: 1579.4725 - val_goal_loss: 0.4280
Epoch 3/200
228/228 [==============================] - 23s 100ms/step - loss: 1582.0581 - position_loss: 1579.4034 - goal_loss: 0.6028 - val_loss: 1546.7672 - val_position_loss: 1544.2184 - val_goal_loss: 0.4153
Epoch 4/200
228/228 [==============================] - 24s 104ms/step - loss: 852.4290 - position_loss: 847.7030 - goal_loss: 0.3992 - val_loss: 467.0402 - val_position_loss: 461.4209 - val_goal_loss: 0.2808
Epoch 5/200
228/228 [==============================] - 25s 111ms/step - loss: 410.4434 - position_loss: 404.8786 - goal_loss: 0.2779 - val_loss: 296.8687

In [ ]:
import numpy as np
from keras.models import load_model

# Load the previously saved model
model = load_model('/content/sample_data/y_position_goal_model.keras')

# function to generate predictions recursively
def generate_predictions(initial_position, initial_acceleration, num_timesteps, num_predictions):
    current_position = np.full((num_timesteps, 1), initial_position)  # Shape (num_timesteps, 1)
    initial_acceleration = np.array([initial_acceleration])  # Shape (1,)

    predictions = []

    for _ in range(num_predictions):
        input_position = current_position.reshape(1, num_timesteps, 1)  # Reshape for LSTM input
        input_acceleration = initial_acceleration.reshape(1, 1)  # Reshape for LSTM input
        predicted_position = model.predict([input_position, input_acceleration])

        # check the type of predicted_position and ensure it's an np.array
        if isinstance(predicted_position, list):
            predicted_position = np.array(predicted_position[0])
        else:
            predicted_position = np.array(predicted_position)

        # append the predicted position to the output
        predictions.append(predicted_position.flatten().tolist())

        # update the current position sequence
        current_position = np.vstack((current_position[1:], predicted_position))

    return predictions

#sample initial pos and acc values taken from a training data file to test the model
initial_position = 0.06491089799162905 # Starting x position
initial_acceleration = 3.200573950646076  # Starting ax value, assuming constant acceleration
num_timesteps = 13
num_predictions = 10

# generate predictions
predicted_positions = generate_predictions(initial_position, initial_acceleration, num_timesteps, num_predictions)

# print the predicted positions
for i, position in enumerate(predicted_positions, 1):
    print(f"Predicted position {i}: y = {position}")

1/1 [==============================] - 0s 44ms/step
Predicted position 1: y = [4.4802446365356445]
Predicted position 2: y = [9.563464164733887]
Predicted position 3: y = [14.889108657836914]
Predicted position 4: y = [19.449068069458008]
Predicted position 5: y = [23.859079360961914]
Predicted position 6: y = [28.993919372558594]
Predicted position 7: y = [33.685787200927734]
Predicted position 8: y = [37.86956024169922]
Predicted position 9: y = [42.03355407714844]
Predicted position 10: y = [46.58573913574219]
